In this notebook, we show how to run the decision-focused learning algorithms SFGE [1] and LANCER [2] on the Weighted-Set Multi Cover problem.

In [ ]:
import os
import sys

path_to_project = os.path.dirname(os.path.abspath("")) + "/"
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("decision-focused-learning-codebase"))))

In [1]:
import inspect

from src.concrete_models.grbpy_two_stage_weighted_set_multi_cover import (
    WeightedSetMultiCover,
)
from src.decision_makers.differentiable_decision_maker import (
    DifferentiableDecisionMaker,
)
from src.decision_makers.lancer_decision_maker import LancerDecisionMaker
from src.decision_makers.sfge_decision_maker import SFGEDecisionMaker
from src.generate_data_functions.generate_data_wsmc import gen_data_wsmc
from src.problem import Problem
from src.runner import Runner

(CVXPY) Aug 21 03:33:06 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.14.6206). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Aug 21 03:33:06 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.14.6206). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')


Auto-Sklearn cannot be imported.


The Weighted Set Multi-Cover Problem with Recourse Actions is characterized as follows:

- There is a set of elements that need to be covered by a collection of sets.
- Each set covers a subset of the elements.
- Each element has a coverage requirement, specifying how many times it must be covered.
- Each set has a cost, and the objective is to select sets to minimize the total cost while ensuring all coverage requirements are met.

We look specifically at the two-stage variant of the problem, where:
- In the first-stage, the sets are selected, while the coverage requirements are unknown.
- The true coverage requirements are revealed in the second-stage.
- A recourse action allows corrects for the items that were not covered, and assigns a penalty based on the missed items.
- A special variant of the problem has also a recourse in the other direction, namely by looking at the number of unused sets.


We now construct and `OptimizationModel` some given parameters (we set the recovery ratio to 0, such that there is only a recourse for uncovered items) and print the settings that are used in the initialization.

In [2]:
model = WeightedSetMultiCover(
    num_items=2,
    num_covers=3,
    penalty=5,
    cover_costs_lb=5,
    cover_costs_ub=50,
    seed=5,
    recovery_ratio=0,
)
print(f"Settings: {inspect.signature(model.__init__)}")

Set parameter Username
Set parameter LicenseID to value 2612263
Academic license - for non-commercial use only - expires 2026-01-20
Set parameter FeasibilityTol to value 1e-06
Settings: (num_items: int, num_covers: int, penalty: float, cover_costs_lb: int, cover_costs_ub: int, recovery_ratio: float = 0, seed: int = 0, silvestri2023: bool = False, density: float = 0.25, num_scenarios: int = 1)


The `seed` parameter and the arguments `cover_costs_lb` and `cover_costs_ub` determine the fixed cover costs parameters that are set in the optimization model. 

In [3]:
print(f"The cover costs for this WSMC are given by {model.cover_costs}")

The cover costs for this WSMC are given by [40. 19. 49.]


The parameters to predict for this problem are the coverage requirements:

In [4]:
print(model.param_to_predict_names)

['coverage_requirements']


We can use a data generation script to generate our train and test data. This data generation script could be replaced by reading the data from a file. 

In [5]:
data_dict = gen_data_wsmc(
    seed=4,
    num_data=500,
    num_features=5,
    num_items=model.num_items,
    degree=1,
    noise_width=0.5,
)
print(data_dict.keys())

dict_keys(['coverage_requirements', 'features'])


In [6]:
print(f"The shape of the features are {data_dict['features'].shape}")
print(f"The shape of the coverage requirements are {data_dict['coverage_requirements'].shape}")

The shape of the features are (500, 5)
The shape of the coverage requirements are (500, 2)


We are making use of a problem class to split the generated data into a training set, a validation set and a test set.

In [7]:
problem = Problem(
    data_dict=data_dict,
    opt_model=model,
    train_ratio=0.4,
    val_ratio=0.2,
    compute_optimal_decisions=True,
    compute_optimal_objectives=True,
)

Computing optimal decisions for the entire dataset...
Optimal decisions computed and added to dataset.
Computing optimal objectives for the entire dataset...
Optimal objectives computed and added to dataset.
Shuffling indices before splitting...
Dataset split completed: Train=200, Validation=100, Test=200


We can access the train/validation and test instances through the problem class:

In [8]:
print(f"The size of the training set equals {problem.train_size}, which corresponds to the number of training indices {len(problem.train_indices)}.")

The size of the training set equals 200, which corresponds to the number of training indices 200.


Now we are almost set to run a DFL algorithm. All DFL algorithms have their own objects, which are subclasses from the base `DecisionMaker` object. We need to pass the `Problem` object to the `DecisionMaker` at initialization. The DecisionMaker also includes the predictive model, for which we pass the arguments by `predictor_str` and `predictor_kwargs`. 

In [9]:
predictor_kwargs = {
    "size": 256,
    "n_layers": 2,
    "activation": "leaky_relu",
    "output_activation": "leaky_relu",
}
predictor_str = "MLP"

These can be passed to the `DecisionMaker`

In [10]:
decision_maker = SFGEDecisionMaker(
    problem=problem,
    learning_rate=0.005,
    batch_size=32,
    device_str="cpu",
    loss_function_str="regret",
    predictor_str=predictor_str,
    predictor_kwargs=predictor_kwargs,
    noisifier_kwargs={"sigma_setting": "independent", "sigma_init": 0.5},
)

Problem mode set to: train
Problem mode set to: train
set learning rate to 0.005


We need use the `Runner` class to run our DFL algorithm

In [11]:
runner = Runner(decision_maker, num_epochs=2, use_wandb=False)
runner.run()

Epoch 0/2: Starting initial validation...
Problem mode set to: validation


/Users/noah/PycharmProjects/decision-focused-learning-codebase/src/problem.py:494: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  losses_tensor = torch.tensor(losses_list)


Epoch Results:
validation/objective_mean: 521.0078
validation/sym_rel_regret_mean: 0.2200
validation/mse_mean: 3.9477
validation/coverage_requirements_mean: 6.2706
validation/abs_regret_mean: 214.7578
validation/x_mean: 2.3333
validation/rel_regret_mean: 0.6631
Initial best validation metric (abs_regret): 214.7578125
Starting training...
Epoch: 1/2
Problem mode set to: train
Epoch Results:
validation/objective_mean: 521.0078
validation/sym_rel_regret_mean: 0.2200
validation/mse_mean: 3.9477
validation/coverage_requirements_mean: 6.2706
validation/abs_regret_mean: 214.7578
validation/x_mean: 2.3333
train/solver_calls_mean: 224.5714
train/loss_mean: -0.0159
train/eval_mean: -0.0000
validation/rel_regret_mean: 0.6631
train/sigma_mean: 0.5016
Problem mode set to: validation
Epoch Results:
validation/objective_mean: 566.4062
validation/sym_rel_regret_mean: 0.2687
validation/mse_mean: 21.5131
validation/coverage_requirements_mean: 8.3747
validation/abs_regret_mean: 260.1562
validation/x_mean

np.float32(214.75781)

We can run the same algorithm with a Linear model (0 hidden layers and identity output activation)

In [12]:
linear_predictor_kwargs = {"n_layers": 0, "output_activation": "identity"}
decision_maker = SFGEDecisionMaker(
    problem=problem,
    learning_rate=0.01,
    batch_size=32,
    device_str="cpu",
    loss_function_str="regret",
    predictor_str=predictor_str,
    predictor_kwargs=linear_predictor_kwargs,
    noisifier_kwargs={"sigma_setting": "independent"},
)
runner = Runner(decision_maker, num_epochs=2, use_wandb=False)
runner.run()

Problem mode set to: train
Problem mode set to: train
Problem mode set to: train
set learning rate to 0.01
Epoch 0/2: Starting initial validation...
Problem mode set to: validation
Epoch Results:
validation/objective_mean: 645.5391
validation/sym_rel_regret_mean: 0.3029
validation/mse_mean: 6.2306
validation/coverage_requirements_mean: 6.4558
validation/abs_regret_mean: 339.2891
validation/x_mean: 2.6224
validation/rel_regret_mean: 1.0440
Initial best validation metric (abs_regret): 339.2890625
Starting training...
Epoch: 1/2
Problem mode set to: train
Epoch Results:
validation/objective_mean: 645.5391
validation/sym_rel_regret_mean: 0.3029
validation/mse_mean: 6.2306
validation/coverage_requirements_mean: 6.4558
validation/abs_regret_mean: 339.2891
validation/x_mean: 2.6224
train/solver_calls_mean: 224.5714
train/loss_mean: -0.2551
train/eval_mean: 0.0000
validation/rel_regret_mean: 1.0440
train/sigma_mean: 1.9537
Problem mode set to: validation
Epoch Results:
validation/objective_mea

np.float32(267.3203)

Suppose we want to compare the SFGE with LANCER algorithm (proposed by Zharmagambetov et al.). To make a fair comparison the predictive model should be the same for both decision makers. Be aware that the LANCER algorithm has many hyperparamters that can be tuned that influence it's performance.

In [13]:
decision_maker = LancerDecisionMaker(
    problem=problem,
    batch_size=32,
    device_str="cpu",
    predictor_str=predictor_str,
    predictor_kwargs=predictor_kwargs,
)
runner = Runner(decision_maker, num_epochs=2, use_wandb=False, experiment_name="lancer")
runner.run()

Problem mode set to: train
Problem mode set to: train
Set learning rate surrogate to 0.001
Set learning rate predictor to 0.005
Epoch 0/2: Starting initial validation...
Problem mode set to: validation
Epoch Results:
validation/objective_mean: 553.7000
validation/sym_rel_regret_mean: 0.2318
validation/mse_mean: 4.0995
validation/coverage_requirements_mean: 6.2278
validation/abs_regret_mean: 241.9900
validation/x_mean: 2.3333
validation/rel_regret_mean: 0.7012
Initial best validation metric (abs_regret): 241.99000549316406
Starting training...
Epoch: 1/2
Problem mode set to: train
Epoch Results:
validation/objective_mean: 553.7000
validation/sym_rel_regret_mean: 0.2318
validation/mse_mean: 4.0995
validation/coverage_requirements_mean: 6.2278
validation/abs_regret_mean: 241.9900
train/predictor_loss_mean: 5.7146
validation/x_mean: 2.3333
train/solver_calls_mean: 300.0000
train/eval_mean: 277.7250
validation/rel_regret_mean: 0.7012
train/surrogate_loss_mean: 164730.1250
Problem mode set t

np.float32(241.99)

And, an evaluation would not be complete without a comparison against prediction-focused-learning, i.e. training the predictive model on the MSE. We can do so by running:


In [16]:
decision_maker = DifferentiableDecisionMaker(
    problem=problem,
    learning_rate=0.001,
    batch_size=32,
    device_str="cpu",
    loss_function_str="mse",
    predictor_str=predictor_str,
    predictor_kwargs=predictor_kwargs,
)
runner = Runner(decision_maker, num_epochs=5, use_wandb=False, experiment_name="pfl")
runner.run()

Problem mode set to: train
Problem mode set to: train
Epoch 0/5: Starting initial validation...
Problem mode set to: validation
Epoch Results:
validation/objective_mean: 521.0078
validation/sym_rel_regret_mean: 0.2200
validation/mse_mean: 3.8879
validation/coverage_requirements_mean: 6.2316
validation/abs_regret_mean: 214.7578
validation/x_mean: 2.3333
validation/rel_regret_mean: 0.6631
Initial best validation metric (abs_regret): 214.7578125
Starting training...
Epoch: 1/5
Problem mode set to: train
Epoch Results:
validation/objective_mean: 521.0078
validation/sym_rel_regret_mean: 0.2200
validation/mse_mean: 3.8879
validation/coverage_requirements_mean: 6.2316
validation/abs_regret_mean: 214.7578
validation/x_mean: 2.3333
train/solver_calls_mean: 100.0000
train/loss_mean: 3.6220
train/grad_norm_mean: 3.4642
validation/rel_regret_mean: 0.6631
Problem mode set to: validation
Epoch Results:
validation/objective_mean: 507.1992
validation/sym_rel_regret_mean: 0.2139
validation/mse_mean: 3.

np.float32(171.13281)

In [17]:
# TODO: add graphs